In [1]:
# made by Gibeom LEE, HI LAB
# 1M to 100k.ipynb 와 달리, folder_path안에 있는 모든 csv파일을 전부 축소 시켜줘서, 다른 폴더에 저장해주는 코드입니다.

In [2]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

In [ ]:
# 원본 CSV 파일이 있는 폴더 경로
folder_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250904_다 찍기\csv파일\물방울 부피 별"

# find Peak 설정
height = 0.05  # 피크 감지 기준 전압 크기 (노이즈 레벨 고려)s
num_samples = 10000 # 최종 줄이고자 하는 행 수

In [4]:
# 결과를 저장할 새 폴더 경로 생성
parent_dir = os.path.dirname(folder_path)
original_folder_name = os.path.basename(folder_path)
new_folder_name = f"minimized_{original_folder_name}"
output_folder = os.path.join(parent_dir, new_folder_name)
os.makedirs(output_folder, exist_ok=True)

# 모든 CSV 파일 경로 수집
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

for file_path in csv_files:
    try:
        # CSV 파일 읽기 (헤더 없음)
        df = pd.read_csv(file_path, header=None, names=["Time", "Voltage"], skiprows=2)

        # 피크 검출
        peaks_positive, _ = find_peaks(df["Voltage"], height=height, distance=100)
        peaks_negative, _ = find_peaks(-df["Voltage"], height=height, distance=100)
        peaks = np.sort(np.concatenate((peaks_positive, peaks_negative)))

        # 피크 개수 확인
        num_peaks = len(peaks)
        print(f"{os.path.basename(file_path)} → Peak count: {num_peaks}")

        if num_peaks >= num_samples:
            selected_indices = np.sort(np.random.choice(peaks, num_samples, replace=False))
        else:
            num_non_peak_samples = num_samples - num_peaks
            non_peak_indices = np.setdiff1d(np.arange(len(df)), peaks)
            selected_non_peaks = np.linspace(0, len(non_peak_indices) - 1, num_non_peak_samples, dtype=int)
            selected_non_peaks = non_peak_indices[selected_non_peaks]

            selected_indices = np.sort(np.concatenate((peaks, selected_non_peaks)))

        # 축소된 데이터 생성
        filtered_df = df.iloc[selected_indices]

        # 새 파일 이름 및 저장 경로
        filename = os.path.basename(file_path)
        save_path = os.path.join(output_folder, filename)
        filtered_df.to_csv(save_path, index=False, header=False)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

C16_5s_000.csv → Peak count: 7900
C16_5s_010.csv → Peak count: 5411
C16_5s_020.csv → Peak count: 5262
C16_5s_030.csv → Peak count: 5247
C16_5s_040.csv → Peak count: 5037
C16_5s_050.csv → Peak count: 5270
C16_5s_100.csv → Peak count: 4794
C16_5s_150.csv → Peak count: 4340
C16_5s_200.csv → Peak count: 4579
C16_60s_000.csv → Peak count: 4567
C16_60s_010.csv → Peak count: 4951
C16_60s_020.csv → Peak count: 4814
C16_60s_030.csv → Peak count: 5108
C16_60s_040.csv → Peak count: 5260
C16_60s_050.csv → Peak count: 5099
C16_60s_100.csv → Peak count: 4904
C16_60s_150.csv → Peak count: 4906
C16_60s_200.csv → Peak count: 5319
C30_5s_000.csv → Peak count: 4491
C30_5s_010.csv → Peak count: 4928
C30_5s_020.csv → Peak count: 5900
C30_5s_030.csv → Peak count: 5372
C30_5s_040.csv → Peak count: 5829
C30_5s_050.csv → Peak count: 5768
C30_5s_100.csv → Peak count: 6145
C30_5s_150.csv → Peak count: 5805
C30_5s_200.csv → Peak count: 5995
C30_60s_000.csv → Peak count: 6095
C30_60s_010.csv → Peak count: 5764
C30